# Python Learning

Using the Boost.Python library, I compiled my tic-tac-toe `Board` class as a library accessable to python 3.7. 

### To Do:
- Added winner variable to TicTacToe class because it's messy to have to keep calling whoWon()
- Develop DeepQAgent class


In [ ]:
import sys
sys.path.append("../lib")

In [ ]:
import numpy as np
import tensorflow as tf
import Board

from pathlib import Path

In [ ]:
class TicTacToe():
    def __init__(self, player1, player2, p1_symbol=5, p2_symbol=-2, p1_exporation=1, p2_exporation=1):  
        # define two players
        p1_name = player1 + str(p1_symbol)
        player1 = globals()[player1]
        self.player1 = player1(symbol=p1_symbol, name=p1_name, exploration=p1_exporation)
        p2_name = player2 + str(p2_symbol)
        player2 = globals()[player2]
        self.player2 = player2(symbol=p2_symbol, name=p2_name, exploration=p2_exporation)
        
        self.turnPlayer = self.player1
                
        # turn couter
        self.moveCounter = 1
        
        # initialize c++ game
        self.game = Board.Board(int(p1_symbol), int(p2_symbol))

    def printMoveMap(self):
        print("----------------")
        print("|  0 |  1 |  2 |")
        print("----------------")
        print("|  3 |  4 |  5 |")
        print("----------------")
        print("|  6 |  7 |  8 |")
        print("----------------")
        
    def printGame(self):
        gameBoard = self.game.getBoard()
        print(gameBoard)     
        
    def play_pvp(self):
        self.printMoveMap()
        
        while not self.game.isBoardFull() and self.game.whoWon() == 0:
            print("Turn: ", self.moveCounter, "Player: ", self.game.getTurnPlayer())
            self.printGame()
            
            move = self.turnPlayer.getMove(self.game)
            if self.game.addMove(move):
                # successfully added move, so change turn player
                if self.moveCounter % 2 == 1:
                    self.turnPlayer = self.player2
                else:
                    self.turnPlayer = self.player1
                    
                self.moveCounter += 1
        
        if self.game.whoWon() != 0:
            if self.player1.symbol == self.game.whoWon():
                print("Player", self.player1.name, "(", self.game.whoWon(), ")", "Won!")
            else:
                print("Player", self.player2.name, "(", self.game.whoWon(), ")", "Won!")
        elif self.game.isBoardFull():
            print("Stalemate...")
        else:
            print("Crazy Error: Uncaught stopping condition!")
        
        print("Final Board:")
        self.printGame()
                

In [ ]:
class Human():
    def __init__(self, symbol, exploration, name):
        self.symbol = symbol
        self.name = name
        self.exploration = exploration
    
    def getMove(self, state):
        validMoves = state.getValidMoves()
        keepTryingInputs = True
        while keepTryingInputs:
            move = int(input('Choose move: '))
            keepTryingInputs = (move not in validMoves)
        
        return move

In [ ]:
class Rand():
    def __init__(self, symbol, exploration, name):
        self.symbol = symbol
        self.exploration = exploration
        self.name = name
    
    def getMove(self, state):
        randMove = int(np.random.choice(state.getValidMoves(), 1))
        return randMove

In [ ]:
class DeepQAgent():
    def __init__(self, symbol, exploration, name):
        self.symbol = symbol
        self.name = name
        self.exploration = exploration
        
        self.model = self.getModel()
        
    def loadModel(self):
        modelPath = Path(self.name+'.h5')
        if modelPath.is_file():
            model = tf.keras.models.load_model(s)
            print('Model loaded:', self.name+'.h5')
        else:
            model = self.buildModel()
        
        return model
    
    def buildModel(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(18, activation='relu', input_shape=(9,)))
        model.add(tf.keras.layers.Dense(18, activation='relu'))
        model.add(tf.keras.layers.Dense(18, activation='linear'))
        model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
        
        return model
    
    def saveModel(self):
        print("saving model (fake)")

In [ ]:
game = TicTacToe('Human', 'Rand')
game.play_pvp()